<a href="https://colab.research.google.com/github/michaelherediaperez/EspecializacionEstructuras/blob/main/01_AnalisisNoLineal/Bhatti_72.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio Bhatti 7.2. Constant Stiffness Iteration.

Michael Heredia Pérez - mherediap@unal.edu.co - Octubre 04 / 2021

Mismo problema 7.1 sin iterar la matriz tangencial.

In [11]:
# Iportación de librerías.
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math

## Parámetros del modelo

Algunos parámetros geométricos son:

In [12]:
nefs = 3        # Número de elementos finitos.
nnds = nefs+1   # Número de nodos.
ngdl = nnds     # Número de grados de libertad.
L = 1           # Rango superior del dominio.

# Posición de los nodos (x1, x, x3, x4)
xnod = np.linspace(0, L, nnds)  

Parámetros de carga

In [13]:
# Carga axial distribuida.
q = 4

# Vector de cargas puntuales.
Rp = np.zeros(ngdl)

Introduciendo las condiciones de frontera

In [14]:
EBC = 1    # Essential Boundary Condition u1 = 1.
NBC = 2    # Non-essential Boundary Condition.

Al igual que en ele jercicio 7.1. se resuelve mediante Newton-Raphson considerando única la matriz de rigidez inicial.

In [15]:
# Vctor inicial (suposición) de desplazamientos d^(0)
d_i = np.array([EBC, 2, 2, 2], dtype=float)   

display(Math(r'd^{(0)} = '))
print(d_i)

<IPython.core.display.Math object>

[1. 2. 2. 2.]


El vector $R_E$ inicial se construye a partir de la NBC:

In [16]:
Re_nbc = np.array([0, 0, 0,  NBC])

display(Math(r'R_{E,NBC} ='))
print(Re_nbc)

<IPython.core.display.Math object>

[0 0 0 2]


Durante el calculo iterativo, ni el vector de fuerzas externas **ni la matriz tangenial** cambiarán, así que se peuden calcular fuera del ciclo principal

In [17]:
# Reservo memoria: 
Re_ = np.zeros(ngdl)         # vector incompleto de fuerzas nodales externas.
Kt  = np.zeros((ngdl, ngdl)) # Matriz tangencial.

# Para cada elemento.
for e in range(nefs):
    
    # Coordenadas globales de los nodos:
    x1 = xnod[e]; x2 = xnod[e+1]
    # Longitud del elemento:
    l = x2-x1
    # Desplazamientos en los nodos locales del elemento.
    u1 = d_i[e]; u2 = d_i[e+1] 
    
    # Matriz tangencial del elemento e en la interación i
    kt_i = np.array([
                    [ u1**2/l, -u2**2/l],
                    [-u1**2/l,  u2**2/l]
                    ])
    
    # Vector de cargas externas local para ele elemento e:
    # re = rq + rp
    re = np.array([ q*l/2, q*l/2 ]) + np.array([ Rp[e], Rp[e+1] ])

    # gdl que interceden.
    idx = np.array([e, e+1])
    
    # Ensamblaje de la matriz tangencial global.
    Kt[np.ix_(idx, idx)] += kt_i    

    # Ensamblaje del vector de fuerzas externas.
    Re_[idx] += re

# Vector de fuerzas externas final.
Re = Re_nbc + Re_

# Simplificaciones debido a la condición EBC.
Kt = np.delete(Kt, 0, axis=0)
Kt = np.delete(Kt, 0, axis=1)
Re = np.delete(Re, 0) 

# Norma de Re al cuadrado.
norm_Re2 = np.sum(Re**2)

In [18]:
display(Math(r'R_{E sin NBC} ='))
print(Re_.round(5))

print("\nVector de fuerzas externas ensamblado")
display(Math(r'R_{E} ='))
print(Re.round(5))

print("\nMatriz tangencial de rigidez inicial")
display(Math(r'K_T^{0} =')); print(Kt.round(4))

<IPython.core.display.Math object>

[0.66667 1.33333 1.33333 0.66667]

Vector de fuerzas externas ensamblado


<IPython.core.display.Math object>

[1.33333 1.33333 2.66667]

Matriz tangencial de rigidez inicial


<IPython.core.display.Math object>

[[ 24. -12.   0.]
 [-12.  24. -12.]
 [  0. -12.  12.]]


Se hace un ciclo iterativo para solucionar el problema:

In [19]:
# Número de iteraciones a realizar:
NIT = 4

# Por cada iteración:
for itr in range(NIT):
    
    # Reservo memoria:    
    Ri = np.zeros(ngdl)         # Vector de fuerzas internas. 

    # Calculo para cada EF.
    for e in range(nefs):

        # Desplazamientos en los nodos locales del elemento.
        u1 = d_i[e]; u2 = d_i[e+1] 
        # Coordenadas globales de los nodos:
        x1 = xnod[e]; x2 = xnod[e+1]
        # Longitud del elemento:
        l = x2-x1
        
        # Vector de fuerzas internas.
        ri_i = np.array([(u1**3 - u2**3)/(3*l), -(u1**3 - u2**3)/(3*l)])
        
        # gdl que interceden.
        idx = np.array([e, e+1])
        
        # Ensamblaje del vector de fuerzas internas.
        Ri[idx] += ri_i

    # Simplificaciones debido a la condición EBC.
    Ri = np.delete(Ri, 0)

    # Calculo del residuo R = Re - Ri
    R = Re-Ri

    # Norma de R al cuadrado.
    norm_R2 = np.sum(R**2)

    # Criterio de convergencia:
    CNR = norm_R2 / (1+norm_Re2)

    # Solución de las ecuacioens incremetales:
    delta_d = np.linalg.solve(Kt, R)

    # Comprobación del CNR.
    d_i[1:] += delta_d

    # Resultados.
    print('-'*80 + "\nIteración ", str(itr+1))
    #display(Math(r'K_T^{} ='.format(itr)));              print(Kt.round(4))
    display(Math(r'R_I^{} ='.format(itr)));              print(Ri.round(5))
    display(Math(r'R^{} ='.format(itr)));                print(R.round(5))
    display(Math(r'\frac{\|R\|^2}{1 + \| R_E\|^2} = ')); print(CNR)
    display(Math(r'd^{} ='.format(itr+1)));              print(d_i.round(5))

--------------------------------------------------------------------------------
Iteración  1


<IPython.core.display.Math object>

[7. 0. 0.]


<IPython.core.display.Math object>

[-5.66667  1.33333  2.66667]


...

[ 0.00387  0.15324 -0.15935]


<IPython.core.display.Math object>

0.004190775532008012


<IPython.core.display.Math object>

[1.      1.8502  2.178   2.34743]
